In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from keras.models import load_model
from tkinter import *
import json
import random
import DataV2 as st

c:\Users\Brahim Laghzaoui\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
lemmatizer = WordNetLemmatizer()
modell = load_model('chatbot_model.h5')
intents = json.loads(open('data.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - split wrds into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of wrds array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, wrds, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of wrds - matrix of N wrds, vocabulary matrix
    bag = [0] * len(wrds)
    for s in sentence_words:
        for i, w in enumerate(wrds):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

def predict_class(sentence, modelll):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    print(p.shape)
    res = modelll.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def get_response(ints, intents_json,msg):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            w = nltk.word_tokenize(msg)
            ner = nltk.pos_tag(w)
            num = 0
            for n in ner :
                if(n[1] == 'CD'):
                    num = n[0]
            if(tag == 'Status' and num == 0) :
                result = "give me the num type"
            if num != 0 :
                if len(num) > 1 :
                    result = st.main(num)
                if len(num) == 1 :
                    result = st.getLinkActivitie(int(num))
            else :
                result = random.choice(i['responses'])
            break
    return result



def chatbot_response(msg):
    ints = predict_class(msg, modell)
    res = get_response(ints, intents,msg)
    return str(res)


In [13]:
from flask import Flask, render_template, request, jsonify
app = Flask(__name__)
app.static_folder = 'static'

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get",methods=['POST'])
def get_bot_response():
    userText = request.get_json().get('message')
    reponse = chatbot_response(userText)
    message = {"answer":reponse}
    return jsonify(message)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Sep/2022 15:50:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 15:50:48] "GET /static/css/chat.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 15:50:48] "GET /static/lib/animate/animate.min.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 15:50:48] "GET /static/lib/owlcarousel/assets/owl.carousel.min.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 15:50:48] "GET /static/lib/lightbox/css/lightbox.min.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 15:50:48] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 15:50:48] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 15:50:48] "GET /static/img/insights.png HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 15:50:48] "GET /static/img/hero.png HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 15:50:48] "GET /static/img/intelligent.png HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 15:50:48] "GET /static/img/career.png HTTP/1.1" 200 -
